In [ ]:
# 크롤링 순서
# 1차 크롤링 : 블로그 조회 -> title / url / date
# 2차 크롤링 : url별 방문하여 해시태그 가져오기
# 2차 크롤링 기준 추가 : 수집된 hashtag가 200개이면 다음 크롤링 진행 

- import

In [ ]:
import os
import sys
import pandas as pd

import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

- url list 

In [ ]:
df = pd.read_excel('../Data/서울시115장소명 목록_장소명수정_20240527.xlsx')
keywords = [i for i in df['AREA_SEARCH']]

In [ ]:
# start_time = timer()

# 검색어 입력
df = pd.read_excel('../Data/서울시115장소명 목록_장소명수정_20240527.xlsx')
keywords =df['AREA_SEARCH']

keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-1)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []
for query in keywords:
    # start_crawling_time = timer()
    # print(start_crawling_time)
    try:
        # 검색 URL 생성
        base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(base_url)
        time.sleep(0.5)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        # search_number = soup.find(class_="search_number")
        # total_num = search_number.text.replace("건",'').replace(",",'')
        # total_num = int(total_num)
        # end_page=total_num//7+1

        # 페이지마다 게시물 가져오기
        for page_num in range(1, 100):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(0.5)  # 페이지가 로드될 때까지 대기

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 블로그 포스트 데이터 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            posts_date_data = soup.select('div.info_post > div.writer_info')

            # 타이틀 수가 7개가 아니면 루프를 빠져나오기
            if len(posts_data) != 1:
                for post, post_date in zip(posts_data, posts_date_data):
                    title = post.text.replace('\n', '').strip()
                    href = post.attrs['href']
                    date = post_date.find('span', 'date').text.strip()
                    try :
                        date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                    except:
                        date_timestamp = endDate
                    crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
            else :
                break
    except Exception as e:
        print(f"에러 발생: {e}")

    # end_crawling_time = timer()
    # print(end_crawling_time)
    # print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))


crawling_blog_data = pd.DataFrame(crawling_blog_data)
file_name = f'../Data/crawling_blog_list_{startDate}-{endDate}.csv'
crawling_blog_data.to_csv(file_name, index=False)

In [ ]:
- hashtag list

In [ ]:
# url list 가져오기
crawling_data = pd.read_csv(file_name)
crawling_data

# location list
df = pd.read_excel('../Data/서울시115장소명 목록_장소명수정_20240527.xlsx')
test_df=[location for location in df['AREA_SEARCH']]

# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)

crawling_contents_data = []

for keyword in keywords:
    temp = crawling_data[crawling_data['location'] == keyword]
    total_tags=0
    for index, row in temp.iterrows():
        url = str(row['url'])
        try:
            driver.get(url)
            # iframe 접근
            WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame')))
            # tag 찾아서 가져오기
            tags_element = driver.find_element(By.CLASS_NAME, "wrap_tag")
            tags = tags_element.text.strip('태그').split('\n')
            tags=[tag for tag in tags if tag != '']
            tags = [tag.replace(keyword,"") for tag in tags if tag != '']
            tags = [tag.replace(keyword,"") for tag in tags if tag != '#']

            # 제외할 단어 리스트 (추가 예정)
            # exclud_list =[]
            # if tags in exclude_list:
            #   continue

            # data 담기
            if len(tags)>0:
                crawling_contents_data.append({"location": keyword, "url": url, "tags": tags})
                total_tags += len(tags)
                print(f'{index}번째 완료: {tags} / 총 {total_tags} 개')
                # 해시태그 개수가 200개 이상이면 다음 키워드로 넘어가기
                if total_tags >= 200:
                    break

        except Exception as e:
            print(f"error ({index}) : {str(e)}")

    # 해시태그 개수가 200개 이상이면 다음 키워드로 넘어가기
    # if total_tags >= 200:
    #     break

driver.quit()

In [ ]:
- 파일저장

In [ ]:
df_tags = pd.DataFrame(crawling_contents_data)
df_tags

In [ ]:
file_name = f'../Data/crawling_tags_data_{startDate}-{endDate}.csv'
df_tags.to_csv(f"{file_name}", index=False)